# Model comparison

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler


df = pd.read_excel("combined_monomers_rdkit_new.xlsx")
df = df.dropna(subset=["Knockdown", "molecular_weight"])
df = df.dropna(axis=1)
df = pd.get_dummies(df, columns=['cell type'])
df.loc[df["Knockdown"] == 1, "Knockdown"] = 0
df.loc[df["Knockdown"] == 3, "Knockdown"] = 2
df.loc[df["Knockdown"] == 2, "Knockdown"] = 1
for col in df.columns:
    if col.endswith('1'):
        df[col] = df[col] * (1 - df["lipophil_w"])
    elif col.endswith('2'):
        df[col] = df[col] * df["lipophil_w"]
X = df.drop(["Polymer Name", "Knockdown"], axis=1)
y = df.Knockdown.astype(int)
accuracies = None
conf_matrices = None
print(y.value_counts())

models = [
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    ExtraTreesClassifier(),
    SVC(),
    MLPClassifier(),
    GaussianNB(),
    KNeighborsClassifier(),
    XGBClassifier(),
    LGBMClassifier(verbose=-1),
]

model_names = [
    'Decision Tree',
    'Random Forest',
    'Extra Trees',
    'SVC',
    'MLP',
    'Naive Bayes',
    'KNN',
    'XGBoost',
    'LightGBM'
]

accuracies = {name: [] for name in model_names}
conf_matrices = {name: [] for name in model_names}

scaler = StandardScaler()
unique_labels = np.unique(y)

for split in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
    #scaler can be switched on or off
    #X_train = scaler.fit_transform(X_train)
    #X_test = scaler.transform(X_test)

    #resampling can be adjusted and switched on or off
    #oversampler =RandomOverSampler(sampling_strategy={1: 500})
    #undersampler = RandomUnderSampler()
    #X_train, y_train = oversampler.fit_resample(X_train, y_train)
    #X_train, y_train = undersampler.fit_resample(X_train, y_train)

    for model, name in zip(models, model_names):
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = balanced_accuracy_score(y_test, y_pred)
        accuracies[name].append(accuracy)
        
        cm = np.zeros((len(unique_labels), len(unique_labels)), dtype=int)
        
        cm_temp = confusion_matrix(y_test, y_pred, labels=unique_labels)
        
        for i, label in enumerate(unique_labels):
            for j, _ in enumerate(unique_labels):
                cm[i, j] = cm_temp[i, j] if i < cm_temp.shape[0] and j < cm_temp.shape[1] else 0
                
        conf_matrices[name].append(cm)

for name in model_names:
    mean_accuracy = np.mean(accuracies[name])
    std_dev_accuracy = np.std(accuracies[name])
    print("{} BAc: {} SDA: {}".format(name, mean_accuracy, std_dev_accuracy))


    avg_cm = np.mean(conf_matrices[name], axis=0)
    print(avg_cm)


max_values = [max(values) for values in accuracies.values()]
print("MAX_acc: ", max_values)


# Hypertuning

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import RandomUnderSampler
from hyperopt import hp, fmin, tpe, Trials

df = pd.read_excel("combined_monomers_rdkit_new.xlsx")
df = df.dropna(subset=["Knockdown", "molecular_weight"])
df = df.dropna(axis=1)
df = pd.get_dummies(df, columns=['cell type'])
df.loc[df["Knockdown"] == 1, "Knockdown"] = 0
df.loc[df["Knockdown"] == 3, "Knockdown"] = 2
df.loc[df["Knockdown"] == 2, "Knockdown"] = 1
for col in df.columns:
    if col.endswith('1'):
        df[col] = df[col] * (1 - df["lipophil_w"])
    elif col.endswith('2'):
        df[col] = df[col] * df["lipophil_w"]
df = df.drop(df.columns[-1], axis=1)
X = df.drop(["Polymer Name", "Knockdown"], axis=1)
y = df.Knockdown.astype(int)

oversampler = SMOTEENN()

space = {
    'num_leaves': hp.choice('num_leaves', range(20, 150)), 
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),  
    'n_estimators': hp.choice('n_estimators', range(100, 1000)),  
    'max_depth': hp.choice('max_depth', range(5, 20)), 
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),  
    'subsample': hp.uniform('subsample', 0.5, 1.0),  
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0)  
}

def objective(params):
    model = LGBMClassifier(
        verbose = -1,
        num_leaves=int(params['num_leaves']),
        learning_rate=params['learning_rate'],
        n_estimators=int(params['n_estimators']),
        max_depth=int(params['max_depth']),
        reg_lambda=params['reg_lambda'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree']
    )
    
    scores = []
    
    for i in range (10):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)
        X_train,y_train = oversampler.fit_resample(X_train,y_train)
        X_train,y_train = undersampler.fit_resample(X_train,y_train)
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        model = model.fit(X_train, y_train)  

        preds = model.predict(X_test)
        score = balanced_accuracy_score(y_test, preds)
        scores.append(score)

    average_score = np.mean(scores)
    
    return -average_score 




trials = Trials()
best_hyperparams = fmin(fn=objective,
                        space=space,
                        algo=tpe.suggest,
                        max_evals=100,
                        trials=trials)

print("The best hyperparameters are : ","\n")
print(best_hyperparams)


# Model stability

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, balanced_accuracy_score
from lightgbm import LGBMClassifier
from imblearn.combine import SMOTEENN

features_to_keep=["BalabanJ1","fr_piperdine1","MaxAbsEStateIndex1","BCUT2D_MRLOW3","BCUT2D_MRLOW1","EState_VSA32","Ipc2","fr_NH01","FpDensityMorgan21","PBF1","VSA_EState91"]    
# Parameters
params = {
    'colsample_bytree': 0.5280489323078961, 'learning_rate': 0.023499558736047456, 
    'max_depth': 8, 'n_estimators': 10, 'num_leaves': 110, 'reg_lambda': 0.8070820186132228, 
    'subsample': 0.8610806429810899
}

# Load and prepare data
df = pd.read_excel("combined_monomers_rdkit_new.xlsx")
df = df.dropna(subset=["Knockdown", "molecular_weight"])
df = df.dropna(axis=1)

#df = pd.get_dummies(df, columns=['cell type'])
df.loc[df["Knockdown"] == 1, "Knockdown"] = 0
df.loc[df["Knockdown"] == 3, "Knockdown"] = 2
df.loc[df["Knockdown"] == 2, "Knockdown"] = 1

for col in df.columns:
    if col.endswith('1'):
        df[col] = df[col] * (1 - df["lipophil_w"])
    elif col.endswith('2'):
        df[col] = df[col] * df["lipophil_w"]


X = X[features_to_keep]
feature_names = X.columns
y = df.Knockdown.astype(int)
# Models to evaluate
#Possible to integrate other basic models as well
models = [
    LGBMClassifier(verbose=-1,**params)
]

model_names = [
    'LightGBM (optimized)'
]

balanced_accuracies = {name: [] for name in model_names}
conf_matrices = {name: [] for name in model_names} 

scaler = StandardScaler()


for split in range(100):
    X_train_o, X_test, y_train_o, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
    X_train_o = scaler.fit_transform(X_train_o)
    X_test=scaler.transform(X_test)

    

    for model, name in zip(models, model_names):
        X_train = X_train_o.copy()
        y_train = y_train_o.copy()
        if name=='LightGBM (optimized)':

            oversampler = SMOTEENN(random_state=1)
            X_train, y_train = oversampler.fit_resample(X_train, y_train)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        balanced_acc = balanced_accuracy_score(y_test, y_pred)
        balanced_accuracies[name].append(balanced_acc)

        cm = confusion_matrix(y_test, y_pred)
        conf_matrices[name].append(cm)

for name in model_names:
    mean_balanced_acc = np.mean(balanced_accuracies[name])
    std_dev_balanced_acc = np.std(balanced_accuracies[name])
    print(f"{name} Balanced Accuracy: {mean_balanced_acc:.4f} (± {std_dev_balanced_acc:.4f})")

    avg_cm = np.mean(conf_matrices[name], axis=0)
    
    plt.figure(figsize=(5, 4))
    sns.heatmap(avg_cm, annot=True, fmt=".2f", cmap="Blues", xticklabels=np.unique(y), yticklabels=np.unique(y))
    plt.title(f'Averaged Confusion Matrix for {name}')
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.show()

mean_balanced_accs = [np.mean(balanced_accuracies[name]) for name in model_names]
std_dev_balanced_accs = [np.std(balanced_accuracies[name]) for name in model_names]

plt.figure(figsize=(15, 6))
print(balanced_accuracies['LightGBM (optimized)'])
plt.plot(balanced_accuracies['LightGBM (optimized)'],marker='o', color='orange')

plt.title('Model Stability')
plt.xlabel('Iteration Number')
plt.ylabel('Balanced Accuracy Score')

plt.tight_layout()
plt.show()

# Compare resampling methods

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, balanced_accuracy_score
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import EditedNearestNeighbours as ENN


params={'colsample_bytree': 0.9755088786798269, 'learning_rate': 0.1827587842746705, 'max_depth': 8, 'n_estimators': 465, 'num_leaves': 85, 'reg_lambda': 0.8324249896997891, 'subsample': 0.9331718683905172}
# Load and prepare data
df = pd.read_excel("combined_monomers_rdkit_new.xlsx")
df = df.dropna(subset=["Knockdown", "molecular_weight"])
df = df.dropna(axis=1)
df = pd.get_dummies(df, columns=['cell type'])
df.loc[df["Knockdown"] == 1, "Knockdown"] = 0
df.loc[df["Knockdown"] == 3, "Knockdown"] = 2
df.loc[df["Knockdown"] == 2, "Knockdown"] = 1

for col in df.columns:
    if col.endswith('1'):
        df[col] = df[col] * (1 - df["lipophil_w"])
    elif col.endswith('2'):
        df[col] = df[col] * df["lipophil_w"]


# Define resampling strategies
resampling_strategies = {
    'without resampling': None,
    'Oversampling': RandomOverSampler(sampling_strategy="auto"),
    'Undersampling': RandomUnderSampler(),
    'SMOTE': SMOTE(sampling_strategy="auto"),
    'SMOTEENN (default)': SMOTEENN(random_state=1),
    'SMOTEENN (auto)': SMOTEENN(smote=SMOTE(sampling_strategy="auto"), enn=ENN(sampling_strategy='auto')),
    'SMOTE + Undersampling (80%)': (SMOTE(sampling_strategy={1: 500}), RandomUnderSampler()),
}

# Initialize variables to store results
balanced_accuracies = {name: [] for name in resampling_strategies}
conf_matrices = {name: [] for name in resampling_strategies}
test_point_results = {name: [] for name in resampling_strategies}
scaler = StandardScaler()

# Model to evaluate
model = LGBMClassifier(verbose=-1, **params)

# Evaluate each resampling strategy
for strategy_name, strategy in resampling_strategies.items():
    for split in range(100):
        X = df.drop(["Polymer Name", "Knockdown","molecular_weight"], axis=1)
        test_point = X.iloc[-1]
        X = X.iloc[:-1] 
        feature_names = X.columns
        y = df.Knockdown.astype(int).iloc[:-1]
        print(f"Starting split number {split+1}/100 for {strategy_name}: ")
      
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        test_point = scaler.transform([test_point])
        
        if isinstance(strategy, tuple):
            smote, undersample = strategy
            X_train, y_train = smote.fit_resample(X_train, y_train)
            X_train, y_train = undersample.fit_resample(X_train, y_train)
        elif strategy is None:
            pass
        else:
            X_train, y_train = strategy.fit_resample(X_train, y_train)
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        # Calculate balanced accuracy
        balanced_acc = balanced_accuracy_score(y_test, y_pred)
        balanced_accuracies[strategy_name].append(balanced_acc)
        
        test_predict = model.predict_proba(test_point)[:, 1]
        test_point_results[strategy_name].append(test_predict)
        
        cm = confusion_matrix(y_test, y_pred)
        conf_matrices[strategy_name].append(cm)

for strategy_name in resampling_strategies:
    mean_balanced_acc = np.mean(balanced_accuracies[strategy_name])
    std_dev_balanced_acc = np.std(balanced_accuracies[strategy_name])
    print(f"{strategy_name} Balanced Accuracy: {mean_balanced_acc:.4f} (± {std_dev_balanced_acc:.4f})")
    
    mean_test_point = np.mean(test_point_results[strategy_name])
    print(f"Avg Test Point Proba: {mean_test_point}")
    
    avg_cm = np.mean(conf_matrices[strategy_name], axis=0)
    
    plt.figure(figsize=(5, 4))
    sns.heatmap(avg_cm, annot=True, fmt=".2f", cmap="Blues", xticklabels=np.unique(y), yticklabels=np.unique(y))
    plt.title(f'Averaged Confusion Matrix for {strategy_name}')
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.show()

mean_balanced_accs = [np.mean(balanced_accuracies[name]) for name in resampling_strategies]
std_dev_balanced_accs = [np.std(balanced_accuracies[name]) for name in resampling_strategies]

plt.figure(figsize=(15, 6))
plt.bar(resampling_strategies.keys(), mean_balanced_accs, yerr=std_dev_balanced_accs, capsize=5, color='skyblue')
plt.title('Resampling Strategy Comparison Based on Balanced Accuracy')
plt.xlabel('Resampling Strategy')
plt.ylabel('Balanced Accuracy Score')
plt.xticks(rotation=45)
plt.savefig('Resampling_Strategy_Comparison_Balanced_Accuracy.png')
plt.show()


# Feature reduction

In [ ]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from lightgbm import LGBMClassifier
from imblearn.combine import SMOTEENN
import umap.umap_ as umap


params={'colsample_bytree': 0.9755088786798269, 'learning_rate': 0.1827587842746705, 'max_depth': 8, 'n_estimators': 465, 'num_leaves': 85, 'reg_lambda': 0.8324249896997891, 'subsample': 0.9331718683905172}

df = pd.read_excel("combined_monomers_rdkit_new.xlsx")
df = df.dropna(subset=["Knockdown", "molecular_weight"])
df = df.dropna(axis=1)

df.loc[df["Knockdown"] == 1, "Knockdown"] = 0
df.loc[df["Knockdown"] == 3, "Knockdown"] = 2
df.loc[df["Knockdown"] == 2, "Knockdown"] = 1

for col in df.columns:
    if col.endswith('1'):
        df[col] = df[col] * (1 - df["lipophil_w"])
    elif col.endswith('2'):
        df[col] = df[col] * df["lipophil_w"]
scaler = StandardScaler()
unique_labels = np.unique(y)

smote_enn = SMOTEENN(random_state=1)
model = LGBMClassifier(verbose=-1, **params)
df["paper"]=df["Polymer Name"].str.split("_",expand=True)[0]
paper=df["paper"].iloc[:-1]
features_to_drop=["paper","Polymer Name", "Knockdown"]
X = df.drop(features_to_drop, axis=1)
X=pd.get_dummies(X,drop_first=False)
print(X.columns)
feature_names=X.columns
y = df.Knockdown.astype(int)

X = scaler.fit_transform(X)
Xresampled, yresampled = smote_enn.fit_resample(X, y)

model.fit(Xresampled, yresampled)

feature_importances = model.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})

top_features = feature_importance_df.sort_values(by='Importance', ascending=False).head(11)
print("Top 11 Most Important Features:")
print(top_features)

top_11_features = top_features['Feature'].values  
X_top11 = df[top_11_features] 
y = df.Knockdown.astype(int)

X_top11 = scaler.fit_transform(X_top11)
X_top11 = pd.DataFrame(X_top11, columns=top_11_features)  

with open("combined_monomers_fingerprints_2024-12-30_11-44-29.pkl","rb") as f:
    combined_df=pickle.load(f)

publication=combined_df['Paper Name']
X_fp = combined_df.drop(['Paper Name'],axis=1)
X_fp = X_fp.dropna(axis=1)
umap_params = {
  'n_neighbors': len(df), 
  'min_dist': 1, 
  'spread': 1, 
  'metric': 'euclidean', 
  'random_state': 1
}

umap_fp=umap.UMAP(n_components=2, **umap_params)
umap_full = umap.UMAP(n_components=2, **umap_params)
umap_reduced = umap.UMAP(n_components=2, **umap_params)

embedding_fp=umap_fp.fit_transform(X_fp)
embedding_full = umap_full.fit_transform(X)
embedding_reduced = umap_reduced.fit_transform(X_top11)

binary_colors = ['red', 'green',"blue"]  

unique_labels = np.unique(paper)
color_map = {label: binary_colors[i % len(binary_colors)] for i, label in enumerate(unique_labels)}

y_color_fp = [color_map[label] for label in publication]
y_colors = [color_map[label] for label in paper]

fig, axes = plt.subplots(1, 3, figsize=(12, 8))

axes[0].scatter(
    embedding_fp[:, 0], 
    embedding_fp[:, 1], 
    c=y_color_fp, 
    edgecolor='k', 
    alpha=0.7
)
axes[0].set_title("UMAP Projection: Morgan Fingerprints")
axes[0].set_xlabel("UMAP 1")
axes[0].set_ylabel("UMAP 2")


axes[1].scatter(
    embedding_full[:, 0], 
    embedding_full[:, 1], 
    c=y_colors,  
    edgecolor='k', 
    alpha=0.7
)
axes[1].set_title("UMAP Projection: Full Features")
axes[1].set_xlabel("UMAP 1")
axes[1].set_ylabel("UMAP 2")

axes[2].scatter(
    embedding_reduced[:, 0], 
    embedding_reduced[:, 1], 
    c=y_colors,  
    edgecolor='k', 
    alpha=0.7
)
axes[2].set_title("UMAP Projection: Reduced Features")
axes[2].set_xlabel("UMAP 1")
axes[2].set_ylabel("UMAP 2")

handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=f"Dataset {label}")
           for label, color in color_map.items()]


plt.tight_layout()
plt.show()


# Predictions and SHAP

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier
from imblearn.combine import SMOTEENN
import shap
import matplotlib.pyplot as plt

params={'colsample_bytree': 0.9755088786798269, 'learning_rate': 0.1827587842746705, 'max_depth': 8, 'n_estimators': 465, 'num_leaves': 85, 'reg_lambda': 0.8324249896997891, 'subsample': 0.9331718683905172}

df = pd.read_excel("combined_monomers_rdkit_new.xlsx")
df = df.dropna(subset=["Knockdown", "molecular_weight"])
df = df.dropna(axis=1)

df.loc[df["Knockdown"] == 1, "Knockdown"] = 0
df.loc[df["Knockdown"] == 3, "Knockdown"] = 2
df.loc[df["Knockdown"] == 2, "Knockdown"] = 1

for col in df.columns:
    if col.endswith('1'):
        df[col] = df[col] * (1 - df["lipophil_w"])
    elif col.endswith('2'):
        df[col] = df[col] * df["lipophil_w"]
scaler = StandardScaler()


df_test=  pd.read_excel("/home/akm/Felix_ML/Moldescr_poly/siRNA_ML/New_revision_polymers/cleaned/test_polymers_new.xlsx")

df_test = df_test.dropna(axis=1)

for col in df_test.columns:
    if col.endswith('1'):
        df_test[col] = df_test[col] * (1 - df_test["lipophil_w"])
    elif col.endswith('2'):
        df_test[col] = df_test[col] * df_test["lipophil_w"]

# Resampling strategy
smote_enn = SMOTEENN(random_state=1)
model = LGBMClassifier(verbose=-1,**params)

features_to_keep=["BalabanJ1","fr_piperdine1","MaxAbsEStateIndex1","BCUT2D_MRLOW3","BCUT2D_MRLOW1","EState_VSA32","Ipc2","fr_NH01","FpDensityMorgan21","PBF1","VSA_EState91"]    
y=df.Knockdown
df_new=df[features_to_keep]
trainset=df_new
valset=df_test[features_to_keep]


X = scaler.fit_transform(trainset)
Xresampled, yresampled = smote_enn.fit_resample(X, y)

model.fit(Xresampled, yresampled)                     

X_test=scaler.transform(valset)

print(model.predict(X_test))

shap_values = shap.TreeExplainer(model).shap_values(Xresampled)

shap.summary_plot(shap_values, Xresampled ,feature_names=features_to_keep)

explainer=shap.TreeExplainer(model,X,feature_names=features_to_keep)

explanation=explainer(X_test)
shap.plots.waterfall(explanation[0])
              